Setp 1:- Importing the liberies that will use throughout the code

In [ ]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import pickle

Downlaoded the stop words in english so can clean the data (reviews)

In [2]:
nltk.download("stopwords")
stop_words = [elements.lower() for elements in set(stopwords.words("english"))]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Imporitng the raw data and sotring it in other variable so change don't affect the raw data file

In [3]:
raw_movies_data = pd.read_csv("IMDB_sample.csv")
moives = raw_movies_data
moives.head()

,Unnamed: 0,review,label
0,18245,This short spoof can be found on Elite's Mille...,0
1,19801,A singularly unfunny musical comedy that artif...,0
2,3222,"An excellent series, masterfully acted and dir...",1
3,6010,The master of movie spectacle Cecil B. De Mill...,1
4,16355,I was gifted with this movie as it had such a ...,0


In [4]:
len(raw_movies_data["review"].index)

7501

Setp 2:- Data Preprocessing 

In [5]:
y = moives["label"]
x = moives.drop(columns=["Unnamed: 0" ,"label"])

In [6]:
x["review"] = x["review"] .str.lower()
x.head()

,review
0,this short spoof can be found on elite's mille...
1,a singularly unfunny musical comedy that artif...
2,"an excellent series, masterfully acted and dir..."
3,the master of movie spectacle cecil b. de mill...
4,i was gifted with this movie as it had such a ...


Replacing special or any charachter expect english words

In [7]:
x.replace("[^a-zA-Z]"," ", regex=True, inplace=True)
x.head()

,review
0,this short spoof can be found on elite s mille...
1,a singularly unfunny musical comedy that artif...
2,an excellent series masterfully acted and dir...
3,the master of movie spectacle cecil b de mill...
4,i was gifted with this movie as it had such a ...


Removing the stop words (common words used in english) from the sentence

In [24]:
def remove_stop (x):
    return ",".join([words for words in str(x).split() if words not in stop_words])

x["review"] = x["review"].apply(lambda x: remove_stop(x))
x.head()

,review
0,"thi,short,spoof,found,elit,millennium,edit,dvd..."
1,"singularli,unfunni,music,comedi,artifici,tri,m..."
2,"excel,seri,master,act,direct,unlov,told,mr,dei..."
3,"master,movi,spectacl,cecil,b,de,mill,goe,west,..."
4,"wa,gift,thi,movi,great,premis,friendship,three..."


Spliting the data into training and testing sets

In [25]:
X_train, X_test, y_train, y_test = train_test_split(x,y,
    train_size=0.8, shuffle = True, random_state=8)

Checking the shape of data (how many columns and rows are there in data set)

In [12]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

Creating list of sentences from the data set so in later steps can creat BOW usind tfidf vectorizer

In [26]:
x_train_array = []
for row in range(0, len(X_train.index)):
    x_train_array.append(",".join(str(x) for x in X_train.iloc[row] ))

In [27]:
x_test_array = []
for row in range (0, len(X_test.index)):
    x_test_array.append(",".join(str(x) for x in X_test.iloc[row]))

Using TFIDF to create BOW

In [28]:
tfidfvector = TfidfVectorizer()
tfidf_train_dataset = tfidfvector.fit_transform(x_train_array)
tfidf_test_dataset = tfidfvector.transform(x_test_array)

Step 3:- Modeling

SVM with tfidf and linear karnel

In [29]:
SVM = svm.SVC(kernel='linear', gamma=1.0, C=1.05)
SVM.fit(tfidf_train_dataset, y_train)
predictions = SVM.predict(tfidf_test_dataset)
matrix=confusion_matrix(y_test,predictions)
print(matrix)
score=accuracy_score(y_test,predictions)
print(score)
report=classification_report(y_test,predictions)
print(report)
kappa = cohen_kappa_score(y_test, predictions)
print(kappa)

[[671  81]
 [ 87 662]]
0.8880746169220519
              precision    recall  f1-score   support

           0       0.89      0.89      0.89       752
           1       0.89      0.88      0.89       749

    accuracy                           0.89      1501
   macro avg       0.89      0.89      0.89      1501
weighted avg       0.89      0.89      0.89      1501

0.7761447627028668


Creating pkl file of the model and vectorixzer so can import and use it in app without writing code for the training

In [24]:
pickle.dump(SVM, open("svm.pkl","wb"))
pickle.dump(tfidfvector,open("tfidf.pkl","wb"))